In [3]:
from astropy.io import fits
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import re

In [ ]:
from simple_veloce_reduction import veloce_reduction_tools, veloce_config, veloce_config

In [ ]:
def scan_directory(veloce_paths, date, config, science_targets=[None]):
    """
    Scan input directory and create an obs_list based on FITS headers.

    Parameters:
    - veloce_paths (object): VelocePaths object containing directory paths.
    - date (str): Date string in 'YYYYMMDD' format pointing to date to be scanned.
    - science_targets (list or None): List of science target names to filter by. If None, include all science targets.
    - config (dict): Configuration dictionary.
    """
    ### TODO: validate data quality
    obs_list = {'flat_red': [], 'flat_green': [], 'flat_blue': [], 'flat_blue_long': [],
                'ARC-ThAr_red': [], 'ARC-ThAr_green': [], 'ARC-ThAr_blue': [],
                'SimThLong': [], 'SimTh': [], 'SimLC': [],
                'dark': [], 'bias': [], 'science': []}
    data_dirs = {'red': 'ccd_3', 'green': 'ccd_2', 'blue': 'ccd_1'}
    # pick which arm to reduce
    if config['arm'] in data_dirs.keys():
        arms = [config['arm']]
    elif config['arm'] == 'all':
        arms = data_dirs.keys()
    else:
        raise ValueError('Unsupported arm')
    for arm in arms:
        obs_dir = os.path.join(veloce_paths.input_dir, date, data_dirs[arm])
        if not os.path.exists(obs_dir):
            # veloce_config.log_and_print(f"Directory {obs_dir} does not exist. Skipping.")
            print(f"Directory {obs_dir} does not exist. Skipping.")
            continue
        fits_files = sorted([f for f in os.listdir(obs_dir) if f.endswith('.fits')])
        for file_name in fits_files:
            fits_path = os.path.join(obs_dir, file_name)

            try:
                with fits.open(fits_path) as hdul:
                    header = hdul[0].header
                    exp_time = header.get('EXPTIME', None)
                    target = header.get('OBJECT', '').strip().replace(' ', '')

                    if target == 'BiasFrame':
                        obs_list['bias'].append(file_name)
                    elif target == 'FlatField-Quartz':
                        if float(exp_time) == 0.1 and arm == 'red':
                            obs_list['flat_red'].append(file_name)
                        elif float(exp_time) == 1.0 and arm == 'green':
                            obs_list['flat_green'].append(file_name)
                        elif float(exp_time) == 10.0 and arm == 'blue':
                            obs_list['flat_blue'].append(file_name)
                        elif float(exp_time) == 60.0 and arm == 'blue':
                            obs_list['flat_blue_long'].append(file_name)
                        else:
                            pass
                            # print(f"[Warning]: Non standard flat exp time = {exp_time} for {file_name}")
                    elif target == 'ARC-ThAr':
                        if float(exp_time) == 15 and arm == 'red':
                            obs_list['ARC-ThAr_red'].append(file_name)
                        elif float(exp_time) == 60 and arm == 'green':
                            obs_list['ARC-ThAr_green'].append(file_name)
                        elif float(exp_time) == 180 and arm == 'blue':
                            obs_list['ARC-ThAr_blue'].append(file_name)
                    elif target == 'SimThLong':
                        obs_list['SimThLong'].append(file_name)
                    elif target.strip() == 'SimTh':
                        obs_list['SimTh'].append(file_name)
                    elif target == 'SimLC':
                        obs_list['SimLC'].append(file_name)
                    elif target == 'DarkFrame':
                        obs_list['dark'].append(file_name)
                    elif target == 'Acquire':
                        pass
                    elif target in science_targets or science_targets[0] is None:
                        obs_list['science'].append([target, file_name])
                    else:
                        pass
            except Exception as e:
                print(f"Error reading {fits_path}: {e}")
                # veloce_config.log_and_print(f"Error reading {fits_path}: {e}")
    if science_targets[0] is None:
        science_targets = list(set([target[0] for target in obs_list['science']]))

    return obs_list, science_targets

In [7]:
def load_log_info(log_path, science_targets, selected_arm, day):
    """
    Loads observation log information and categorizes files based on their type and selected arm.

    This function reads an observation log file and categorizes the entries into different types such as
    'flat', 'dark', 'bias', 'science', and 'wave_calib'. It filters the entries based on the selected
    spectrograph arm and the specified calibration type.

    Parameters:
    - log_path (str): The path to the observation log file.
    - science_targets (list of str): A list of science target names to filter the science observations.
    - selected_arm (str): The spectrograph arm to be processed. Valid values are 'red', 'green', and 'blue'.
    - day (str): The day identifier to construct the file names.
    - calib_type (str): The calibration type to filter the wave calibration observations.

    Returns:
    - dict: A dictionary with keys representing different observation types ('flat_red', 'flat_green', 'flat_blue',
      'dark', 'bias', 'science', 'wave_calib') and values being lists of file names or tuples of target names and file names.

    Note:
    - The function assumes a specific format for the observation log file.
    - The function categorizes flat fields based on their exposure times (0.1s for 'flat_red', 1.0s for 'flat_green',
      and 10.0s for 'flat_blue').
    """
    obs_list = {'flat_red': [], 'flat_green': [], 'flat_blue': [], 'flat_blue_long': [],
                'ARC-ThAr_red': [], 'ARC-ThAr_green': [], 'ARC-ThAr_blue': [],
                'SimThLong': [], 'SimTh': [], 'SimLC': [],
                'dark': [], 'bias': [], 'science': []}
    arms = {'red': 3, 'green': 2, 'blue': 1, 'all': None}
    with open(log_path, 'r') as f:
        lines = f.readlines()
        for line in lines[10:]:
            if line[0:4].isdigit():
                run, arm, target, exp_time = [line.split()[i] for i in [0, 1, 2, 5]]
                file_name = f'{day}{arm}{run}.fits'
                if int(arm) == arms[selected_arm] or selected_arm == 'all':
                    if target.strip() == 'BiasFrame':
                        obs_list['bias'].append(file_name)
                    elif target.strip() == 'FlatField-Quartz':
                        if float(exp_time) == 0.1 and int(arm) == 3:
                            obs_list['flat_red'].append(file_name)
                        elif float(exp_time) == 1.0 and int(arm) == 2:
                            obs_list['flat_green'].append(file_name)
                        elif float(exp_time) == 10.0 and int(arm) == 1:
                            obs_list['flat_blue'].append(file_name)
                        elif float(exp_time) == 60.0 and int(arm) == 1:
                            obs_list['flat_blue_long'].append(file_name)
                        else:
                            pass
                            # print(f"[Warning]: Non standard flat exp time = {exp_time} for {file_name}")
                    elif target.strip() == 'ARC-ThAr':
                        if float(exp_time) == 15 and int(arm) == 3:
                            obs_list['ARC-ThAr_red'].append(file_name)
                        elif float(exp_time) == 60 and int(arm) == 2:
                            obs_list['ARC-ThAr_green'].append(file_name)
                        elif float(exp_time) == 180 and int(arm) == 1:
                            obs_list['ARC-ThAr_blue'].append(file_name)
                    elif target.strip() == 'SimThLong':
                        obs_list['SimThLong'].append(file_name)
                    elif target.strip() == 'SimTh':
                        obs_list['SimTh'].append(file_name)
                    elif target.strip() == 'SimLC':
                        obs_list['SimLC'].append(file_name)
                    elif target.strip() == 'DarkFrame':
                        obs_list['dark'].append(file_name)
                    elif target.strip() in science_targets or science_targets is None:
                        obs_list['science'].append([target.strip(), file_name])
                    else:
                        pass
                    
    return obs_list

In [ ]:
def load_run_logs(science_targets, arm, veloce_paths):
    # Define the regular expression pattern for YYMMDD format
    date_pattern = re.compile(r'^\d{6}$')

    # Use list comprehension to filter and sort directories
    dates = sorted(
        [item for item in os.listdir(veloce_paths.input_dir)
         if os.path.isdir(os.path.join(veloce_paths.input_dir, item)) and date_pattern.match(item)])
    
    days = [veloce_config.format_date(date) for date in dates]

    obs_list = {'flat_red': {}, 'flat_green': {}, 'flat_blue': {}, 'flat_blue_long': {},
                'ARC-ThAr_red': {}, 'ARC-ThAr_green': {}, 'ARC-ThAr_blue': {},
                'SimThLong': {}, 'SimTh': {}, 'SimLC': {},
                'dark': {}, 'bias': {}, 'science': {}}
    
    for day, date in zip(days, dates):
        log_path = os.path.join(veloce_paths.input_dir, date)
        log_name = [name for name in os.listdir(log_path) if name.split('.')[-1] == 'log'][0]
        log_path = os.path.join(log_path, log_name)
        temp_obs_list = load_log_info(log_path, science_targets, arm, day)
        for key in temp_obs_list:
            obs_list[key][date] = temp_obs_list[key]

    return obs_list

In [ ]:
def load_night_logs(date, science_targets, arm, veloce_paths):
    day = veloce_config.format_date(date)

    obs_list = {'flat_red': {}, 'flat_green': {}, 'flat_blue': {}, 'flat_blue_long': {},
                'ARC-ThAr_red': {}, 'ARC-ThAr_green': {}, 'ARC-ThAr_blue': {},
                'SimThLong': {}, 'SimTh': {}, 'SimLC': {},
                'dark': {}, 'bias': {}, 'science': {}}
    
    log_path = os.path.join(veloce_paths.input_dir, date)
    log_name = [name for name in os.listdir(log_path) if name.split('.')[-1] == 'log'][0]
    log_path = os.path.join(log_path, log_name)
    temp_obs_list = load_log_info(log_path, science_targets, arm, day)
    for key in temp_obs_list:
        obs_list[key][date] = temp_obs_list[key]

    return obs_list

In [1]:
run = "CSV_Run"
# date = '230826'
# arm = 'green'
# amplifier_mode = 4
# filename = '26aug20146.fits'
# target = 'HD20203-USQ01'

In [11]:
config = {
    # primary information for the reduction
    'reduce': 'run',  # what to reduce options: 'run', 'night', or 'file'
    'date': '230826',  # date of the observation (YYMMDD), if 'reduce' is 'night' or 'file'
    'filename': '26aug20146.fits',  # name of the file to reduce, if 'reduce' is 'file'
    'science_targets': 'targets.dat',  # file containing a list of science targets
    'calib_type': 'SimLC',  # Wavelength calibration to use SimLC, SimThXe (not implemented), Interpolate_lc (LC not implemented), Interpolate_th (Th, not implemented), or Static
    # secondary information for the reduction
    'arm': 'green',  # Which arm to reduce 'all', 'red', 'green' or 'blue'
    'amplifier_mode': 4,  # 2 or 4
    'sim_calib': True,  # Flag indicating if the simultaneous calibration was used True or False
    'plot_diagnostic': True,  # True or False
    'scattered_light': False,  # try to remove scattered light (background) True or False
    'flat_field': False,  # these are fiber flats (white light fiber spectrum) used for pixel sensitivity True or False

    # part responsible for the paths
    'input_dir': 'Data/Raw/CSV_Run',  # path to the directory with data (the input directory should include subdirectories for nights)
    'output_dir': 'Data/Extracted/Step-by-step',  # path to the directory where extracted data will be stored (i.e. output directory)

    # below are paths where code will store intermediate files and diagnostics
    # by default they are set as subdirectories of output directory with extracted data
    'master_dir': 'Default',  # directory for master images for night/run
    'wavelength_calibration_dir': 'Default',  # directory for wavelength calibration related files for the night/run
    'trace_shift_dir': 'Default',  # TODO: add to the code
    'plot_dir': 'Default',  # directory for diagnostic plots

    # below are paths where the code will look for internal files
    # leave as default unless you know what you are doing
    'trace_dir': 'Default',  # path to the directory with traces to be extracted
    'trace_file': 'Default',  # filename of the file with traces to be extracted
    'wave_dir': 'Default',  # path to the directory with reference wavelength solution
}
science_targets = veloce_config.load_target_list(config['science_targets'])

In [8]:
veloce_paths = veloce_config.VelocePaths.from_config(config)

In [ ]:
obs_list = veloce_config.load_night_logs(
            config['date'], config['science_targets'], config['arm'], veloce_paths)

In [17]:
log_path = os.path.join(veloce_paths.input_dir, '230826')
log_filename = [name for name in os.listdir(log_path) if name.split('.')[-1] == 'log'][0]
log_path = os.path.join(log_path, log_filename)
old_obs_list = load_log_info(log_path, science_targets, config['arm'], '26aug')

In [18]:
old_obs_list

{'flat_red': [],
 'flat_green': ['26aug20023.fits',
  '26aug20024.fits',
  '26aug20025.fits',
  '26aug20026.fits',
  '26aug20027.fits',
  '26aug20028.fits',
  '26aug20029.fits',
  '26aug20030.fits',
  '26aug20031.fits',
  '26aug20032.fits',
  '26aug20033.fits',
  '26aug20034.fits',
  '26aug20035.fits',
  '26aug20036.fits',
  '26aug20037.fits',
  '26aug20205.fits',
  '26aug20206.fits',
  '26aug20207.fits',
  '26aug20208.fits',
  '26aug20209.fits',
  '26aug20210.fits',
  '26aug20211.fits',
  '26aug20212.fits',
  '26aug20213.fits',
  '26aug20214.fits',
  '26aug20215.fits',
  '26aug20216.fits',
  '26aug20217.fits',
  '26aug20218.fits',
  '26aug20219.fits'],
 'flat_blue': [],
 'flat_blue_long': [],
 'ARC-ThAr_red': [],
 'ARC-ThAr_green': ['26aug20060.fits',
  '26aug20061.fits',
  '26aug20062.fits',
  '26aug20063.fits',
  '26aug20064.fits',
  '26aug20065.fits',
  '26aug20066.fits',
  '26aug20264.fits',
  '26aug20265.fits',
  '26aug20266.fits',
  '26aug20267.fits',
  '26aug20268.fits',
  '26a

In [45]:
new_obs_list, new_science_targets = scan_directory(veloce_paths, config['date'], config, science_targets=[None])

In [47]:
new_science_targets

['136504',
 '5018.01-CSV03',
 'V2552Oph-CSV01',
 'HIP94235-USQ02',
 '4652.01-CVS03',
 'DSTuc-USQ03',
 'HD20203-USQ01',
 'HD137613-CSV01',
 'HD201557-CSV05',
 '5003.01-CSV03',
 '209952']

In [48]:
from pprint import pprint

def compare_obs_lists(old_obs_list, new_obs_list):
    keys = set(old_obs_list.keys()).union(new_obs_list.keys())
    for key in keys:
        old = old_obs_list.get(key, None)
        new = new_obs_list.get(key, None)
        if old != new:
            print(f"Difference in '{key}':")
            print("  old_obs_list:")
            pprint(old)
            print("  new_obs_list:")
            pprint(new)
            print("-" * 40)

compare_obs_lists(old_obs_list, new_obs_list)

Difference in 'science':
  old_obs_list:
[['HD20203-USQ01', '26aug20146.fits'],
 ['HD20203-USQ01', '26aug20147.fits'],
 ['HD20203-USQ01', '26aug20148.fits'],
 ['HD20203-USQ01', '26aug20149.fits'],
 ['HD20203-USQ01', '26aug20150.fits'],
 ['HD20203-USQ01', '26aug20151.fits'],
 ['HD20203-USQ01', '26aug20152.fits'],
 ['HD20203-USQ01', '26aug20153.fits'],
 ['HD20203-USQ01', '26aug20154.fits'],
 ['HD20203-USQ01', '26aug20155.fits'],
 ['HD20203-USQ01', '26aug20156.fits'],
 ['HD20203-USQ01', '26aug20157.fits'],
 ['HD20203-USQ01', '26aug20158.fits'],
 ['HD20203-USQ01', '26aug20159.fits'],
 ['HD20203-USQ01', '26aug20160.fits'],
 ['HD20203-USQ01', '26aug20161.fits'],
 ['HD20203-USQ01', '26aug20162.fits'],
 ['HD20203-USQ01', '26aug20163.fits']]
  new_obs_list:
[['136504', '26aug20097.fits'],
 ['136504', '26aug20098.fits'],
 ['136504', '26aug20099.fits'],
 ['HD137613-CSV01', '26aug20101.fits'],
 ['HIP94235-USQ02', '26aug20107.fits'],
 ['V2552Oph-CSV01', '26aug20110.fits'],
 ['V2552Oph-CSV01', '26au